In [27]:
import sage.matroids.matroid
from operator import xor
import functools
import random

In [28]:
def iOrder(a,b,t=1):
    # a <_t b, return negative
    # a >_t b, return positive
    
    if a == b: return 0
    if xor((a < t),(b < t)):
        return -(a-b)
    return a-b

In [29]:
def galeHelper(I,order=1):
    I.sort()
    if order == 1: return I
    
    result=[]
    for i in range(len(I)):
        if I[i] >= order:
            result = I[i:] + I[:i]
            break
    if len(result) == 0: return I
    return result

In [30]:
def galeOrder(I,J,order=1):
    # non-comparable, return None
    # I <=_order J under Gale order, return negative
    # I >=_order J, return positive
    # I = J, return 0

    if len(I) == 0 or len(J) == 0:
        return 0
    
    t = order
    
    I = galeHelper(I, order)
    J = galeHelper(J, order)
    
    flag = iOrder(I[0],J[0],t)
    
    for j in range(len(I)):
        # compare I,J one by one
        current = iOrder(I[j],J[j],t)

        if current == 0:
            continue #doesn't change whatever we had
            
        if flag == 0:
            flag = current
            
        if flag != 0 and xor(flag < 0, current < 0):
            return None
        
        
    return flag

In [31]:
class DecoratedPermutation:
    def __init__(self, p, u=[], d=[]):
        self.p = Permutation(p)
        self.fp = self.p.fixed_points()
        if not (set(u+d) <= set(self.fp)):
            print("decorations are not fixed points!!!")
        self.u = u
        self.d = d
    def perm(self):
        return self.p
    def up(self):
        return self.u
    def down(self):
        return self.d
    def fixed_points(self):
        return self.fp
    def info(self):
        print(self.p, self.u, self.d)

In [32]:
def dpToGrNeck(dp):
    n = len(dp.perm())
    grNeck = {}
    for r in range(1,n+1):
        current = []
        for i in (list(range(r,n+1))+list(range(1,r))):
            pi = dp.perm()
            if (iOrder(i,pi(i),r) < 0) or (i in dp.up()):
                current.append(i)
        grNeck[r] = current
    return grNeck

def grNeckToDp(grNeck):
    n = len(grNeck)
    p = [0]*(n+1)
    u = []
    d = []
    for i in range(1,n):
        if set(grNeck[i+1]) != set(grNeck[i]):
#             print(grNeck[i], grNeck[i+1])
            j = set(grNeck[i+1]+grNeck[i]) - set(grNeck[i+1]).intersection(set(grNeck[i]))
            j = list(j - set({i}))[0]
            p[j] = i
        else:
            p[i] = i
            if i in grNeck[i]:
                u.append(i)
            else:
                d.append(i)
                
    left = list(set(range(1,n+1)) - set(p))[0]
    p[n] = left
    if left == n:
        if left in grNeck[n]:
            u.append(left)
        else:
            d.append(left)

    p = p[1:]

    return DecoratedPermutation(p,u,d)

In [33]:
def grNeckToBases(grNeck):
    n = len(grNeck)
    k = len(grNeck[1])
    bases = []

    for B in Subsets(n,k).list():
        B = B.list()
        flag = 0
        for t in range(1,n+1):
            
            if galeOrder(grNeck[t], B, t) == None or galeOrder(grNeck[t], B, t) > 0:
                flag = 1
                break
        if flag == 0: bases.append(B)
    return bases

In [34]:
class Positroid(sage.matroids.matroid.Matroid):
    def __init__(self, n, dp=None, grNeck=None):
        self.E = set(range(1,n+1))
        self.n = n
        
        if dp != None:
            self.dp = dp
            self.grNeck = dpToGrNeck(dp)
        if grNeck != None:
            self.dp = grNeckToDp(grNeck)
            self.grNeck = grNeck
            
        self.B = grNeckToBases(self.grNeck)
            
    def groundset(self):
        return self.E
    def _rank(self, X):
        M = Matroid(groundset = self.E, bases = self.B)
        return M.rank(X)
        
        
    

In [35]:
def allFlats(M):
    # param: M, a matroid
    # return: all flats of possible rank
    
    if not M.is_valid():
        print("Your matroid is not valid.")
        return
    
    r = M.full_rank()
    flats = []
    
    for i in range(0,r+1):
        flats += M.flats(i)

    return flats

In [36]:
def quotient(N,M):
    # return true if N is a quotient of M
    
    # choice: the way of determing quotient
    
    Nflats = allFlats(N)
    Mflats = allFlats(M)
    
    for n in Nflats:
        if n not in Mflats:
            return False
    
    return True
    

In [37]:
def matroidInfo(M):
    # print bases, circuits, and flats of matroid M
    
    print("bases of this matroid: \n", [sorted(b) for b in M.bases()])
    print("circuits of this matroid: \n", [sorted(c) for c in M.circuits()])
    print("flats of this matroid: \n",[sorted(f) for f in allFlats(M)])

In [194]:
for b in Positroid(6, dp=DecoratedPermutation([5,3,2,1,4,6],[],[6])).bases():
    print(list(b))

[1, 2, 4, 6]
[1, 2, 5, 6]
[1, 3, 4, 6]
[1, 3, 5, 6]
[2, 4, 5, 6]
[3, 4, 5, 6]


In [12]:
grNeck = dpToGrNeck(DecoratedPermutation([1,2,3,4,5,6],[],[1,2,3,4,5,6]))
print(grNeck)
dp = grNeckToDp(grNeck)
print(dp.perm(),dp.up(),dp.down())


{1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
[1, 2, 3, 4, 5, 6] [] [1, 2, 3, 4, 5, 6]


In [25]:
grNeck = dpToGrNeck(DecoratedPermutation([5,3,2,1,4,6],[],[6]))
dp = grNeckToDp(grNeck)
print(dp.perm(),dp.up(),dp.down())

[5, 3, 2, 1, 4, 6] [] [6]


In [26]:
P = Positroid(6, dp = DecoratedPermutation([1,2,3,4,5,6],[],[1,2,3,4,5,6]))
matroidInfo(P)

bases of this matroid: 
 [[]]
circuits of this matroid: 
 [[2], [3], [1], [6], [5], [4]]
flats of this matroid: 
 [[1, 2, 3, 4, 5, 6]]


In [37]:
P = Positroid(3, dp = DecoratedPermutation([1,2,3],[],[1,2,3]))
matroidInfo(P)

bases of this matroid: 
 [[]]
circuits of this matroid: 
 [[3], [2], [1]]
flats of this matroid: 
 [[1, 2, 3]]


In [198]:
for b in Positroid(6, grNeck=grNeck).bases():
    print(list(b))

[1, 2, 4, 6] [2, 4, 5, 6]
[2, 4, 5, 6] [3, 4, 5, 6]
[3, 4, 5, 6] [4, 5, 6, 2]
[4, 5, 6, 2] [5, 6, 1, 2]
[5, 6, 1, 2] [6, 1, 2, 4]
[1, 2, 4, 6]
[1, 2, 5, 6]
[1, 3, 4, 6]
[1, 3, 5, 6]
[2, 4, 5, 6]
[3, 4, 5, 6]


In [199]:
P = Positroid(3,dp = DecoratedPermutation([1,2,3],[3],[1,2]))
print([list(l) for l in allFlats(P)])
print([list(l) for l in P.B])
P2 = Positroid(3,dp = DecoratedPermutation([2,1,3],[3],[]))
print([list(l) for l in allFlats(P2)])
print([list(l) for l in P2.B])

[[3], [1, 3], [2, 3], [1, 2, 3]]
[[1, 2]]
[[3], [1, 2, 3]]
[[1], [2]]


In [23]:
pos5 = [[],[],[],[],[],[]]
n = 5
for s in SymmetricGroup(n):
    s = Permutation(s)
    lenFixed = len(s.fixed_points())
    fp = s.fixed_points()
    if lenFixed == 0:
        P = Positroid(n, dp = DecoratedPermutation(s))
        r = P.rank()
        pos5[r].append(P)
        continue
    for k in range(lenFixed+1):
        choose = Subsets(list(range(lenFixed)), k)
        for c in choose:
            u = [fp[i] for i in c]
            d = list(set(fp) - set(u))
            P = Positroid(n, dp = DecoratedPermutation(s, u, d))
            r = P.rank()
            pos5[r].append(P)

In [25]:
posMid = pos5[1:-1]

for P in posMid[-2]:

    save = []
    for i,e in enumerate(posMid[-3]):
        if quotient(e,P):
            save.append(e)
            
#     save2 = []
#     P2 = save[0]
#     for i,e in enumerate(posMid[-3]):
#         if quotient(e, P2):
#             save2.append(e)
    print("=====")
    P.dp.info()
    print("--")
    for s in save:
        s.dp.info()

=====
[1, 2, 3, 4, 5] [1, 2, 3] [4, 5]
--
[1, 2, 3, 4, 5] [1, 2] [3, 4, 5]
[1, 2, 3, 4, 5] [1, 3] [2, 4, 5]
[1, 2, 3, 4, 5] [2, 3] [1, 4, 5]
[2, 1, 3, 4, 5] [3] [4, 5]
[3, 2, 1, 4, 5] [2] [4, 5]
[1, 3, 2, 4, 5] [1] [4, 5]
[2, 3, 1, 4, 5] [] [4, 5]
=====
[1, 2, 3, 4, 5] [1, 2, 4] [3, 5]
--
[1, 2, 3, 4, 5] [1, 2] [3, 4, 5]
[1, 2, 3, 4, 5] [1, 4] [2, 3, 5]
[1, 2, 3, 4, 5] [2, 4] [1, 3, 5]
[2, 1, 3, 4, 5] [4] [3, 5]
[4, 2, 3, 1, 5] [2] [3, 5]
[1, 4, 3, 2, 5] [1] [3, 5]
[2, 4, 3, 1, 5] [] [3, 5]
=====
[1, 2, 3, 4, 5] [1, 2, 5] [3, 4]
--
[1, 2, 3, 4, 5] [1, 2] [3, 4, 5]
[1, 2, 3, 4, 5] [1, 5] [2, 3, 4]
[1, 2, 3, 4, 5] [2, 5] [1, 3, 4]
[2, 1, 3, 4, 5] [5] [3, 4]
[5, 2, 3, 4, 1] [2] [3, 4]
[1, 5, 3, 4, 2] [1] [3, 4]
[2, 5, 3, 4, 1] [] [3, 4]
=====
[1, 2, 3, 4, 5] [1, 3, 4] [2, 5]
--
[1, 2, 3, 4, 5] [1, 3] [2, 4, 5]
[1, 2, 3, 4, 5] [1, 4] [2, 3, 5]
[1, 2, 3, 4, 5] [3, 4] [1, 2, 5]
[1, 2, 4, 3, 5] [1] [2, 5]
[3, 2, 1, 4, 5] [4] [2, 5]
[3, 2, 4, 1, 5] [] [2, 5]
[4, 2, 3, 1, 5] [3] [2, 5]
=====
[1

=====
[3, 2, 5, 4, 1] [2] [4]
--
[5, 2, 3, 4, 1] [2] [3, 4]
[3, 2, 5, 4, 1] [] [2, 4]
[3, 2, 1, 4, 5] [2] [4, 5]
[3, 5, 1, 4, 2] [] [4]
[3, 5, 2, 4, 1] [] [4]
[1, 2, 5, 4, 3] [2] [1, 4]
[3, 1, 5, 4, 2] [] [4]
[5, 2, 1, 4, 3] [2] [4]
=====
[3, 2, 5, 4, 1] [4] [2]
--
[5, 2, 3, 4, 1] [4] [2, 3]
[3, 2, 5, 4, 1] [] [2, 4]
[4, 2, 5, 3, 1] [] [2]
[3, 2, 1, 4, 5] [4] [2, 5]
[3, 2, 5, 1, 4] [] [2]
[4, 2, 5, 1, 3] [] [2]
[1, 2, 5, 4, 3] [4] [1, 2]
[5, 2, 1, 4, 3] [4] [2]
=====
[1, 4, 2, 5, 3] [1] []
--
[1, 5, 2, 3, 4] [1] []
[4, 5, 2, 1, 3] [] []
[5, 4, 2, 1, 3] [] []
[1, 4, 2, 5, 3] [] [1]
[1, 2, 3, 5, 4] [1] [2, 3]
[1, 5, 2, 4, 3] [1] [4]
[4, 1, 2, 5, 3] [] []
[1, 4, 2, 3, 5] [1] [5]
=====
[5, 3, 1, 4, 2] [4] []
--
[5, 4, 1, 2, 3] [] []
[5, 3, 1, 4, 2] [] [4]
[1, 3, 2, 4, 5] [4] [1, 5]
[5, 1, 2, 4, 3] [4] []
[5, 4, 1, 3, 2] [] []
[5, 1, 3, 4, 2] [4] [3]
[5, 3, 1, 2, 4] [] []
[5, 2, 1, 4, 3] [4] [2]
=====
[2, 4, 1, 5, 3] [] []
--
[4, 5, 1, 2, 3] [] []
[5, 4, 1, 2, 3] [] []
[2, 5, 1, 3, 4] [] []

=====
[3, 4, 5, 2, 1] [] []
--
[4, 5, 1, 2, 3] [] []
[5, 4, 1, 2, 3] [] []
[1, 4, 5, 2, 3] [] [1]
[3, 5, 1, 2, 4] [] []
[4, 2, 5, 3, 1] [] [2]
[3, 1, 5, 2, 4] [] []
[4, 1, 5, 2, 3] [] []
[4, 5, 2, 3, 1] [] []
[3, 4, 1, 2, 5] [] [5]
[5, 4, 2, 3, 1] [] []
[3, 5, 2, 4, 1] [] [4]
[4, 5, 3, 2, 1] [] [3]
=====
[1, 5, 2, 4, 3] [1, 4] []
--
[1, 2, 3, 4, 5] [1, 4] [2, 3, 5]
[1, 5, 2, 3, 4] [1] []
[5, 1, 2, 4, 3] [4] []
[1, 5, 2, 4, 3] [1] [4]
[1, 5, 2, 4, 3] [4] [1]
=====
[2, 5, 1, 4, 3] [4] []
--
[2, 1, 3, 4, 5] [4] [3, 5]
[2, 5, 1, 3, 4] [] []
[5, 1, 2, 4, 3] [4] []
[1, 5, 2, 4, 3] [4] [1]
[2, 5, 1, 4, 3] [] [4]
[5, 2, 1, 4, 3] [4] [2]
=====
[4, 3, 5, 2, 1] [] []
--
[4, 5, 1, 2, 3] [] []
[4, 2, 5, 3, 1] [] [2]
[5, 3, 2, 4, 1] [] [4]
[4, 1, 5, 2, 3] [] []
[4, 5, 2, 3, 1] [] []
[4, 3, 1, 2, 5] [] [5]
[5, 3, 1, 2, 4] [] []
[1, 3, 5, 2, 4] [] [1]
[4, 5, 3, 2, 1] [] [3]
=====
[3, 2, 4, 1, 5] [2] [5]
--
[1, 2, 4, 3, 5] [2] [1, 5]
[3, 1, 4, 2, 5] [] [5]
[3, 2, 1, 4, 5] [2] [4, 5]
[4, 2, 1, 3, 5] [2]

=====
[3, 5, 2, 4, 1] [4] []
--
[3, 1, 2, 4, 5] [4] [5]
[5, 2, 3, 4, 1] [4] [2, 3]
[4, 5, 2, 1, 3] [] []
[3, 5, 2, 1, 4] [] []
[5, 1, 2, 4, 3] [4] []
[1, 5, 2, 4, 3] [4] [1]
[4, 5, 2, 3, 1] [] []
[3, 5, 2, 4, 1] [] [4]
=====
[1, 2, 5, 4, 3] [1, 2] [4]
--
[1, 2, 3, 4, 5] [1, 2] [3, 4, 5]
[1, 5, 2, 4, 3] [1] [4]
[2, 5, 1, 4, 3] [] [4]
[1, 2, 5, 4, 3] [1] [2, 4]
[1, 2, 5, 4, 3] [2] [1, 4]
[2, 1, 5, 4, 3] [] [4]
[5, 2, 1, 4, 3] [2] [4]
=====
[1, 2, 5, 4, 3] [1, 4] [2]
--
[1, 2, 3, 4, 5] [1, 4] [2, 3, 5]
[1, 2, 5, 3, 4] [1] [2]
[1, 2, 5, 4, 3] [1] [2, 4]
[1, 2, 5, 4, 3] [4] [1, 2]
[5, 2, 1, 4, 3] [4] [2]
=====
[1, 2, 5, 4, 3] [2, 4] [1]
--
[1, 2, 3, 4, 5] [2, 4] [1, 3, 5]
[1, 2, 5, 3, 4] [2] [1]
[1, 5, 2, 4, 3] [4] [1]
[1, 2, 5, 4, 3] [2] [1, 4]
[1, 2, 5, 4, 3] [4] [1, 2]
=====
[2, 1, 5, 4, 3] [4] []
--
[2, 1, 3, 4, 5] [4] [3, 5]
[2, 1, 5, 3, 4] [] []
[5, 1, 2, 4, 3] [4] []
[1, 2, 5, 4, 3] [4] [1, 2]
[2, 1, 5, 4, 3] [] [4]
=====
[4, 5, 3, 2, 1] [3] []
--
[5, 2, 3, 4, 1] [3] [2, 4]
[5, 1, 3,

In [15]:
pos4 = [[],[],[],[],[]]
n = 4
for s in SymmetricGroup(n):
    s = Permutation(s)
    lenFixed = len(s.fixed_points())
    fp = s.fixed_points()
    if lenFixed == 0:
        P = Positroid(n, dp = DecoratedPermutation(s))
        r = P.rank()
        pos4[r].append(P)
        continue
    for k in range(lenFixed+1):
        choose = Subsets(list(range(lenFixed)), k)
        for c in choose:
            u = [fp[i] for i in c]
            d = list(set(fp) - set(u))
            P = Positroid(n, dp = DecoratedPermutation(s, u, d))
            r = P.rank()
            pos4[r].append(P)

In [21]:
posMid = pos4[1:-1]

for P in posMid[-1]:

    save = []
    for i,e in enumerate(posMid[-2]):
        if quotient(e,P):
            save.append(e)
            
    save2 = []
    P2 = save[0]
#     P2 = Positroid(4, dp = DecoratedPermutation([3,4,1,2],[],[]))
    for i,e in enumerate(posMid[-3]):
        if quotient(e, P2):
            save2.append(e)
    print("=====")
    P.dp.info()
    print("--")
    for s in save:
        s.dp.info()

=====
[1, 2, 3, 4] [1, 2, 3] [4]
--
[1, 2, 3, 4] [1, 2] [3, 4]
[1, 2, 3, 4] [1, 3] [2, 4]
[1, 2, 3, 4] [2, 3] [1, 4]
[2, 3, 1, 4] [] [4]
[2, 1, 3, 4] [3] [4]
[1, 3, 2, 4] [1] [4]
[3, 2, 1, 4] [2] [4]
=====
[1, 2, 3, 4] [1, 2, 4] [3]
--
[1, 2, 3, 4] [1, 2] [3, 4]
[1, 2, 3, 4] [1, 4] [2, 3]
[1, 2, 3, 4] [2, 4] [1, 3]
[2, 4, 3, 1] [] [3]
[2, 1, 3, 4] [4] [3]
[4, 2, 3, 1] [2] [3]
[1, 4, 3, 2] [1] [3]
=====
[1, 2, 3, 4] [1, 3, 4] [2]
--
[1, 2, 3, 4] [1, 3] [2, 4]
[1, 2, 3, 4] [1, 4] [2, 3]
[1, 2, 3, 4] [3, 4] [1, 2]
[3, 2, 4, 1] [] [2]
[1, 2, 4, 3] [1] [2]
[4, 2, 3, 1] [3] [2]
[3, 2, 1, 4] [4] [2]
=====
[1, 2, 3, 4] [2, 3, 4] [1]
--
[1, 2, 3, 4] [2, 3] [1, 4]
[1, 2, 3, 4] [2, 4] [1, 3]
[1, 2, 3, 4] [3, 4] [1, 2]
[1, 3, 4, 2] [] [1]
[1, 2, 4, 3] [2] [1]
[1, 3, 2, 4] [4] [1]
[1, 4, 3, 2] [3] [1]
=====
[1, 3, 4, 2] [1] []
--
[3, 4, 1, 2] [] []
[1, 3, 4, 2] [] [1]
[1, 4, 2, 3] [1] []
[1, 2, 4, 3] [1] [2]
[4, 3, 1, 2] [] []
[1, 3, 2, 4] [1] [4]
[3, 1, 4, 2] [] []
[1, 4, 3, 2] [1] [3]
=====
[2, 4

In [20]:
P = Positroid(4,dp = DecoratedPermutation([1, 3, 2, 4],[1, 4],[]))
P2 = Positroid(4, dp = DecoratedPermutation([4,1,2,3],[],[]))
print("Necklace is \n", P.grNeck)
matroidInfo(P)
print("====")
print("Necklace is \n", P2.grNeck)
matroidInfo(P2)

Necklace is 
 {1: [1, 2, 3], 2: [2, 3, 4], 3: [1, 3, 4], 4: [1, 3, 4]}
bases of this matroid: 
 [[1, 2, 3], [1, 3, 4], [2, 3, 4]]
circuits of this matroid: 
 [[1, 2, 4]]
flats of this matroid: 
 [[], [1], [2], [3], [4], [1, 2, 4], [1, 3], [2, 3], [3, 4], [1, 2, 3, 4]]
====
Necklace is 
 {1: [1], 2: [2], 3: [3], 4: [4]}
bases of this matroid: 
 [[1], [2], [3], [4]]
circuits of this matroid: 
 [[3, 4], [1, 4], [2, 3], [1, 2], [2, 4], [1, 3]]
flats of this matroid: 
 [[], [1, 2, 3, 4]]


In [39]:
P = Positroid(6,dp = DecoratedPermutation([3,6,1,2,4,5],[],[]))
P2 = Positroid(6, dp = DecoratedPermutation([3,6,1,5,4,2],[],[]))
print("Necklace is \n", P.grNeck)
matroidInfo(P)
print("====")
print("Necklace is \n", P2.grNeck)
matroidInfo(P2)

Necklace is 
 {1: [1, 2], 2: [2, 3], 3: [3, 4], 4: [1, 4], 5: [1, 5], 6: [1, 6]}
bases of this matroid: 
 [[1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [2, 3], [2, 4], [2, 5], [2, 6], [3, 4], [3, 5], [3, 6]]
circuits of this matroid: 
 [[1, 3, 5], [4, 6], [2, 3, 4], [1, 2, 4], [1, 3, 6], [4, 5], [2, 3, 6], [1, 2, 5], [5, 6], [2, 3, 5], [1, 2, 3], [1, 2, 6], [1, 3, 4]]
flats of this matroid: 
 [[], [1], [2], [3], [4, 5, 6], [1, 2, 3, 4, 5, 6]]
====
Necklace is 
 {1: [1, 2, 4], 2: [2, 3, 4], 3: [3, 4, 6], 4: [1, 4, 6], 5: [1, 5, 6], 6: [1, 4, 6]}
bases of this matroid: 
 [[1, 2, 4], [1, 2, 5], [1, 3, 4], [1, 3, 5], [1, 4, 6], [1, 5, 6], [2, 3, 4], [2, 3, 5], [2, 4, 6], [2, 5, 6], [3, 4, 6], [3, 5, 6]]
circuits of this matroid: 
 [[1, 3, 6], [4, 5], [2, 3, 6], [1, 2, 3], [1, 2, 6]]
flats of this matroid: 
 [[], [1], [2], [3], [4, 5], [6], [1, 2, 3, 6], [1, 4, 5], [2, 4, 5], [3, 4, 5], [4, 5, 6], [1, 2, 3, 4, 5, 6]]


In [29]:
pos3 = [[],[],[],[]]
n = 3
for s in SymmetricGroup(n):
    s = Permutation(s)
    lenFixed = len(s.fixed_points())
    fp = s.fixed_points()
    if lenFixed == 0:
        P = Positroid(n, dp = DecoratedPermutation(s))
        r = P.rank()
        pos3[r].append(P)
        continue
    for k in range(lenFixed+1):
        choose = Subsets(list(range(lenFixed)), k)
        for c in choose:
            u = [fp[i] for i in c]
            d = list(set(fp) - set(u))
            P = Positroid(n, dp = DecoratedPermutation(s, u, d))
            r = P.rank()
            pos3[r].append(P)

In [32]:
posMid = pos3[1:-1]

for P in posMid[-1]:

    save = []
    for i,e in enumerate(posMid[-2]):
        if quotient(e,P):
            save.append(e)
        
    print("=====")
    P.dp.info()
    print("--")
    for s in save:
        s.dp.info()

=====
[1, 2, 3] [1, 2] [3]
--
[1, 2, 3] [1] [2, 3]
[1, 2, 3] [2] [1, 3]
[2, 1, 3] [] [3]
=====
[1, 2, 3] [1, 3] [2]
--
[1, 2, 3] [1] [2, 3]
[1, 2, 3] [3] [1, 2]
[3, 2, 1] [] [2]
=====
[1, 2, 3] [2, 3] [1]
--
[1, 2, 3] [2] [1, 3]
[1, 2, 3] [3] [1, 2]
[1, 3, 2] [] [1]
=====
[2, 3, 1] [] []
--
[3, 1, 2] [] []
[1, 3, 2] [] [1]
[3, 2, 1] [] [2]
[2, 1, 3] [] [3]
=====
[1, 3, 2] [1] []
--
[1, 2, 3] [1] [2, 3]
[3, 1, 2] [] []
[1, 3, 2] [] [1]
=====
[3, 2, 1] [2] []
--
[1, 2, 3] [2] [1, 3]
[3, 1, 2] [] []
[3, 2, 1] [] [2]
=====
[2, 1, 3] [3] []
--
[1, 2, 3] [3] [1, 2]
[3, 1, 2] [] []
[2, 1, 3] [] [3]


In [267]:
P1 = Positroid(4, dp = DecoratedPermutation([1,4,3,2],[],[1,3]))
P2 = Positroid(4, dp = DecoratedPermutation([4,2,3,1],[],[2,3]))
P1.is_isomorphic(P2)
matroidInfo(P1)

bases of this matroid: 
 [[1, 2, 3], [1, 3, 4]]
circuits of this matroid: 
 [[2, 4]]
flats of this matroid: 
 [[], [1], [2, 4], [3], [1, 2, 4], [1, 3], [2, 3, 4], [1, 2, 3, 4]]


In [266]:
P = Positroid(4, dp = DecoratedPermutation([3,4,1,2],[],[]))
matroidInfo(P)

bases of this matroid: 
 [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
circuits of this matroid: 
 [[1, 2, 3], [2, 3, 4], [1, 3, 4], [1, 2, 4]]
flats of this matroid: 
 [[], [1], [2], [3], [4], [1, 2, 3, 4]]


In [268]:
P = Positroid(4, dp = DecoratedPermutation([4,3,2,1],[],[]))
matroidInfo(P)

bases of this matroid: 
 [[1, 2], [1, 3], [2, 4], [3, 4]]
circuits of this matroid: 
 [[1, 4], [2, 3]]
flats of this matroid: 
 [[], [1, 4], [2, 3], [1, 2, 3, 4]]


In [262]:
matroidInfo(P1)
matroidInfo(P2)

bases of this matroid: 
 [[1, 2, 3], [1, 3, 4]]
circuits of this matroid: 
 [[2, 4]]
flats of this matroid: 
 [[], [1], [2, 4], [3], [1, 2, 4], [1, 3], [2, 3, 4], [1, 2, 3, 4]]
bases of this matroid: 
 [[1, 2, 3], [2, 3, 4]]
circuits of this matroid: 
 [[1, 4]]
flats of this matroid: 
 [[], [1, 4], [2], [3], [1, 2, 4], [1, 3, 4], [2, 3], [1, 2, 3, 4]]


In [258]:
count = 0
for s in SymmetricGroup(7):
    s = Permutation(s)
    if len(s.fixed_points()) == 0:
        count+=1
count

1854

In [255]:
for P2 in save:
    save2 = []
    P2 = Positroid(4, dp=DecoratedPermutation([1, 2, 4, 3],[1],[2]))
    for i,e in enumerate(posMid[-3]):
        if quotient(e, P2):
            save2.append(e)
    print("===")
    P2.dp.info()
    print("----")
    for s in save2:
        s.dp.info()

===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []
===
[1, 2, 4, 3] [1] [2]
----
[1, 2, 3, 4] [1, 3, 4] [2]
[1, 3, 4, 2] [1] []
[1, 2, 4, 3] [1, 2] []


In [243]:
print(len(posMid[-3]), len(save2))
print(save2)

15 11
[4, 5, 6, 7, 14, 10, 11, 8, 9, 13, 12]


In [231]:
matroidInfo(P)
P.dp.info()

bases of this matroid: 
 [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
circuits of this matroid: 
 [[1, 2, 3, 4]]
flats of this matroid: 
 [[], [1], [2], [3], [4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3, 4]]
[4, 1, 2, 3] [] []


In [232]:
len(save)

11

In [245]:
for s in save:
#     print("==")
#     matroidInfo(s)
    s.dp.info()

AttributeError: 'int' object has no attribute 'dp'

In [235]:
for s in save2:
    print("=====")
#     matroidInfo(s)
    s.dp.info()

=====
[1, 3, 4, 2] [1] []
=====
[2, 4, 3, 1] [3] []
=====
[3, 2, 4, 1] [2] []
=====
[2, 3, 1, 4] [4] []
=====
[2, 3, 4, 1] [] []


In [122]:
pos4 = pos4[1:-1]
m = len(pos4)
result = []
for i in reversed(range(1,len(pos4))):
    matr = []
    for j in range(len(pos4[i])):
        P = pos4[i][j]
        current = [0]*len(pos4[i-1])
        for k in range(len(pos4[i-1])):
            C = pos4[i-1][k]
            if quotient(C,P):
                current[k] = 1
        print(current)
        matr.append(current)
    result.append(matr)

[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [123]:
result

[[[1,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0],
  [1,
   0,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   1,
   0],
  [0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0],
  [0,
   0,
   1,
   0,
   1,
   1,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1],
  [0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   1,
   0],
  [0,